In [1]:
import sys
import pandas as pd
import cv2 as cv
import numpy as np
from tqdm.notebook import tqdm
from skimage.feature import local_binary_pattern
import os

sys.path.append(os.path.join(os.path.dirname(os.getcwd()),"modules"))

In [2]:
import NbrRegionSegment as seg
from MarinePredator.MPA import MPA

In [3]:
def get_fit_oneimg(name, wr, wg, wc, thres):
    #read the image
    img = cv.imread("../Dataset/" + name + ".jpg")

    gray_img = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

    #LBP part
    radius = 1
    points = 8*radius 

    lbp = local_binary_pattern(gray_img, points, radius, "uniform").astype("uint8")

    #segment image
    r = seg.RegionExtract(img, lbp, thres, wr, wg, wc)
    r1, r2 = r

    #find the fitness level
    intra, inter = seg.GetAllSegVariance(gray_img, r1, r2.shape[0])

    return intra+inter

In [4]:
#fitness function to be used in the marine predator algo
def fitness_func(wr, wg, wc, thres):
    #read the csv file
    data = pd.read_csv("../CSV/reduced.csv")

    #fitness for each image against each predator
    fit = list()

    #fetch the image file
    for entry in tqdm(data.values):
        #push the fitness into the list
        fit.append(get_fit_oneimg(entry[2], wr, wg, wc, thres)) #-> fitness is the sum of the inter and intra variance and is a decreasing function
    
    #return the fitness as the avg of all fitness
    return np.mean(fit)

In [5]:
fitness_func(.4, .4, .8, 25)

  0%|          | 0/11 [00:00<?, ?it/s]

In [25]:
#algo params for the MPA
max_iter = 30
num_agents = 15

#wr, wg, wc, thres
lb = [0, 0, 0, 10]
ub = [0.99999999, 0.999999999, 0.99999999, 50]
num_var = 4

#create a class object for the metaheuristic algorithm
mpa = MPA(num_agents, max_iter, lb, ub, num_var, fitness_func)

In [26]:
mpa.initialize()

In [27]:
mpa.Prey.shape

(15, 4)

In [28]:
mpa.Prey

array([[4.09944929e-01, 7.24153550e-01, 7.87419025e-02, 2.16452419e+01],
       [9.01224219e-01, 7.03071358e-01, 5.87718352e-01, 4.10079369e+01],
       [3.81660722e-01, 8.65584517e-01, 2.63323210e-01, 3.72495122e+01],
       [4.01581909e-01, 6.42775697e-01, 3.78658190e-01, 1.29760582e+01],
       [2.79910128e-02, 2.63431140e-01, 4.98000218e-01, 4.04047372e+01],
       [3.41538991e-01, 9.64314499e-01, 4.80354571e-01, 3.11421683e+01],
       [5.41290179e-01, 1.57548981e-01, 2.54270787e-01, 4.10453151e+01],
       [5.13445535e-01, 5.52147222e-01, 6.56910238e-01, 4.04915527e+01],
       [4.14895140e-01, 6.99042552e-01, 2.68776431e-01, 1.62842159e+01],
       [8.42075019e-01, 2.50189492e-02, 1.94806385e-01, 4.55934840e+01],
       [5.17266182e-02, 2.68002723e-01, 7.96557635e-01, 1.38576732e+01],
       [4.41488679e-01, 8.46848455e-01, 9.64286802e-01, 3.93221697e+01],
       [3.88765659e-01, 1.79258616e-01, 8.09817048e-01, 3.30219186e+01],
       [3.93545093e-02, 2.52661154e-01, 4.60619870e

In [29]:
for i in tqdm(mpa.iter_gen(), total = mpa.maxItr):
    pass

  0%|          | 0/30 [00:00<?, ?it/s]

Fitness Calculation:   0%|          | 0/11 [00:00<?, ?it/s]